## 1.	模型简介
C3D模型广泛用于3D视觉任务。C3D网络的构造类似于常见的2D卷积网，主要区别在于C3D使用像卷积3D这样的3D操作，而2D卷积网则是通常的2D架构。要了解有关C3D网络的更多信息，您可以阅读原始论文学习3D卷积网络的时空特征。

3D卷积图示：

![在这里插入图片描述](https://img-blog.csdnimg.cn/3deabf777b584ab289cd286e687bcc88.png)


深度学习在图像领域的成功应用产生了多个优秀预训练特征提取模型。提取的特征基本都是网络后面FC层的激活值，并且在迁移学习任务上表现良好。但是，基于图像训练的深度特征因为没有编码运动特征，并不适合用在视频上。本文就提出了一种可以学习spatial-temporal特征的深度3D ConvNet。虽然3D卷积并不是本文首次提出，但是本文基于大规模的监督训练数据集和深度网络架构，在不同视频分析任务中本文都取得了不错的效果。C3D提取出的特征将视频中物体信息、场景信息和动作信息都隐式编码进特征，使得不需要根据特定任务进行finetune都可以取得不错的效果。并且C3D拥有上述提到的好的视频特征描述子应具有的四个特征。

## 2.模型结构
C3D网络有8个卷积，5个最大池化和2个完全连接的层，然后是softmax输出层。所有 3D 卷积核都是 3 × 3 × 3，在空间和时间维度上都步幅为 1。3D 池化图层表示为从池 1 到池5。所有池化内核均为 2 × 2 × 2，但池 1 为 1 × 2 × 2。每个完全连接的层有 4096 个输出单元。

## 网络结构代码
C3D功能函数说明:
数据预处理：
VideoResize：改变输入视频大小
VideoRescale：此运算符将使用给定的重新缩放和移位来重新缩放输入视频帧。output=image*rescale+shift重新缩放输入视频。
VideoRandomCrop：在随机位置裁剪给定的视频序列（t x h x w x c）
VideoRandomHorizontalFlip：以给定的概率翻转视频的每一帧
VideoReOrder：重新排列数据的维度顺序

参数：
in_d：输入数据的深度，它可以被视为视频的帧数。默认值：16。
in_h：输入帧的高度。默认值：112。
in_w：输入帧的宽度。默认值：112。
in_channel（int）：输入数据的通道数。默认值：3。
kernel_size（Union[int，Tuple[int]]）：C3D中每个conv3d层的卷积核大小。
默认值：（3，3，3）。
head_channel（Tuple[int]）：两个全连接层大小。默认值：[4096，4096]。
num_classes（int）：类的数量，它是每个样本的分类得分大小，即：math：`CLASSES_{out}`。默认值：400。
keep_prob（Tuple[int]）：multi-dense-layer头部的dropout概率，概率数等于multi-dense-layer的数量。
pretrained（bool）：如果为“True”，它将创建一个预训练模型，预训练模型将被加载
从网络。如果为“False”，它将创建一个c3d模型，并对权重和偏差进行统一初始化。
Inputs:
Tensor of shape :math:`(N, C_{in}, D_{in}, H_{in}, W_{in})`.
Outputs:
Tensor of shape :math:`(N, CLASSES_{out})`.


## 执行案例

In [35]:
from mindspore import nn
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor
from mindspore.nn.loss import SoftmaxCrossEntropyWithLogits
from mindspore.nn.metrics import Accuracy
from mindvideo.data.transforms import VideoRandomCrop, VideoRescale, VideoResize, VideoReOrder, VideoRandomHorizontalFlip, VideoCenterCrop

from mindvideo.utils.check_param import Validator,Rel


##### 数据集加载

通过基于VideoDataset编写的UCF101类来加载UCF101数据集。

In [36]:
from mindvideo.data.ucf101 import UCF101

dataset = UCF101(path='/home/publicfile/UCF101-dataset/data',
                    split="train",
                    seq=16,
                    num_parallel_workers=4,
                    shuffle=True,
                    batch_size=16,
                    repeat_num=1,
                    )
ckpt_save_dir = './c3d'

##### 数据处理

用VideoShortEdgeResize根据短边来进行Resize，再用VideoRandomCrop对Resize后的视频进行随机裁剪，再用VideoRandomHorizontalFlip根据概率对视频进行水平翻转，通过VideoRescale对视频进行缩放，利用VideoReOrder对维度进行变换，再用VideoNormalize进行归一化处理。

In [37]:
from mindvideo.data.transforms import VideoNormalize, VideoShortEdgeResize, VideoReOrder,VideoCenterCrop, VideoRescale, VideoResize, VideoReOrder,VideoRandomCrop, VideoRandomHorizontalFlip, VideoRescale

transforms = [VideoResize([128, 171]),
              VideoRescale(shift="/home/chenjy/zjut_mindvideo/tutorials/classification/c3d/resized_mean_sports1m.npy"),
              VideoCenterCrop([112, 112]),
              VideoReOrder([3, 0, 1, 2])]

dataset.transform = transforms
dataset_train = dataset.run()
Validator.check_int(dataset_train.get_dataset_size(), 0, Rel.GT)
step_size = dataset_train.get_dataset_size()

[WARNING] ME(10084:139623264462656,MainProcess):2023-03-21-09:05:25.224.891 [mindspore/dataset/engine/datasets_user_defined.py:766] GeneratorDataset's num_parallel_workers: 4 is too large which may cause a lot of memory occupation (>85%) or out of memory(OOM) during multiprocessing. Therefore, it is recommended to reduce num_parallel_workers to 1 or smaller.
[WARNING] ME(10084:139623264462656,MainProcess):2023-03-21-09:05:26.305.706 [mindspore/dataset/core/validator_helpers.py:804] 'Compose' from mindspore.dataset.transforms.py_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Compose' from mindspore.dataset.transforms instead.


##网络构建

In [38]:
from mindvideo.models.c3d import C3D
network= C3D(num_classes=101)

In [39]:
from mindvideo.schedule.lr_schedule import warmup_cosine_annealing_lr_v1

learning_rate = warmup_cosine_annealing_lr_v1(lr=0.0125,
                                            steps_per_epoch=step_size,
                                            warmup_epochs=35,
                                            max_epoch=100,
                                            t_max=100,
                                            eta_min=0)

In [40]:
network_opt = nn.SGD(network.trainable_params(),
                    learning_rate,
                    momentum=0.9,
                    weight_decay=0.00005)

network_loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")

In [41]:
ckpt_config = CheckpointConfig(
        save_checkpoint_steps=step_size,
        keep_checkpoint_max=10)
ckpt_callback = ModelCheckpoint(prefix='c3d',
                                directory=ckpt_save_dir,
                                config=ckpt_config)

In [42]:
# Init the model.
model = Model(network,
              loss_fn=network_loss,
              optimizer=network_opt,
              metrics={"Accuracy": Accuracy()})

In [43]:
# Begin to train.
print('[Start training `{}`]'.format('c3d_ucf101'))
print("=" * 80)
model.train(100,
            dataset_train,
            callbacks=[ckpt_callback, LossMonitor()],
            dataset_sink_mode=False)
print('[End of training `{}`]'.format('c3d_ucf101'))


[WARNING] ME(10084:139623264462656,MainProcess):2023-03-21-09:05:35.621.443 [mindspore/dataset/engine/datasets_user_defined.py:766] GeneratorDataset's num_parallel_workers: 4 is too large which may cause a lot of memory occupation (>85%) or out of memory(OOM) during multiprocessing. Therefore, it is recommended to reduce num_parallel_workers to 1 or smaller.


[Start training `c3d_ucf101`]


[WARNING] ME(10084:139623264462656,MainProcess):2023-03-21-09:05:36.585.360 [mindspore/dataset/core/validator_helpers.py:804] 'Compose' from mindspore.dataset.transforms.py_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Compose' from mindspore.dataset.transforms instead.
[WARNING] ME(10084:139623264462656,MainProcess):2023-03-21-09:05:36.599.742 [mindspore/dataset/engine/datasets_user_defined.py:766] GeneratorDataset's num_parallel_workers: 4 is too large which may cause a lot of memory occupation (>85%) or out of memory(OOM) during multiprocessing. Therefore, it is recommended to reduce num_parallel_workers to 1 or smaller.
[WARNING] ME(10084:139623264462656,MainProcess):2023-03-21-09:05:37.650.361 [mindspore/dataset/core/validator_helpers.py:804] 'Compose' from mindspore.dataset.transforms.py_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Compose' from mindspore.dataset.transforms instead.


epoch: 1 step: 1, loss is 4.615086078643799
epoch: 1 step: 2, loss is 4.615209102630615
epoch: 1 step: 3, loss is 4.615176677703857
epoch: 1 step: 4, loss is 4.615026950836182
epoch: 1 step: 5, loss is 4.615065574645996
epoch: 1 step: 6, loss is 4.615143775939941
epoch: 1 step: 7, loss is 4.615076541900635
epoch: 1 step: 8, loss is 4.615212440490723
epoch: 1 step: 9, loss is 4.615204811096191
epoch: 1 step: 10, loss is 4.615096569061279
epoch: 1 step: 11, loss is 4.61519193649292


KeyboardInterrupt: 

## 评估流程


In [45]:
from mindspore import context
from mindspore.train.callback import Callback

class PrintEvalStep(Callback):
    """ print eval step """
    def step_end(self, run_context):
        """ eval step """
        cb_params = run_context.original_args()
        print("eval: {}/{}".format(cb_params.cur_step_num, cb_params.batch_num))

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")


In [46]:
from mindvideo.data.ucf101 import UCF101

dataset_eval = UCF101(path="/home/publicfile/UCF101-dataset/data",
                          split="val",
                          seq=16,
                          num_parallel_workers=8,
                          shuffle=False,
                          batch_size=16,
                          repeat_num=1,
                      )

In [47]:
from mindvideo.data.transforms import VideoReOrder, VideoRescale, VideoNormalize
from mindvideo.data.transforms import VideoCenterCrop, VideoShortEdgeResize

transforms = [VideoResize([128, 171]),
              VideoRescale(shift="/home/chenjy/zjut_mindvideo/tutorials/classification/c3d/resized_mean_sports1m.npy"),
              VideoCenterCrop([112, 112]),
              VideoReOrder([3, 0, 1, 2])]
dataset_eval.transform = transforms
dataset_eval = dataset_eval.run()

In [48]:
from mindspore import nn
from mindspore.train import Model
from mindspore.nn.loss import SoftmaxCrossEntropyWithLogits
from mindspore import load_checkpoint, load_param_into_net
from mindvideo.models.c3d import C3D

network = C3D(num_classes=101)


# Define loss function.
network_loss = SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")

# Load pretrained model.
param_dict = load_checkpoint(ckpt_file_name='/home/chenjy/c3d.ckpt')
load_param_into_net(network, param_dict)

# Define eval_metrics.
eval_metrics = {'Loss': nn.Loss(),
                'Top_1_Accuracy': nn.Top1CategoricalAccuracy(),
                'Top_5_Accuracy': nn.Top5CategoricalAccuracy()}
print_cb = PrintEvalStep()

# Init the model.
model = Model(network, loss_fn=network_loss, metrics=eval_metrics)

In [49]:
# Begin to eval.
print('[Start eval `{}`]'.format('c3d_ucf101'))
result = model.eval(dataset_eval,
                    callbacks=[print_cb],
                    dataset_sink_mode=False)
print(result)


[WARNING] ME(10084:139623264462656,MainProcess):2023-03-21-09:06:09.490.838 [mindspore/train/model.py:1077] For PrintEvalStep callback, {'step_end'} methods may not be supported in later version, Use methods prefixed with 'on_train' or 'on_eval' instead when using customized callbacks.
[WARNING] ME(10084:139623264462656,MainProcess):2023-03-21-09:06:09.499.595 [mindspore/dataset/engine/datasets_user_defined.py:766] GeneratorDataset's num_parallel_workers: 8 is too large which may cause a lot of memory occupation (>85%) or out of memory(OOM) during multiprocessing. Therefore, it is recommended to reduce num_parallel_workers to 1 or smaller.


[Start eval `c3d_ucf101`]


[WARNING] ME(10084:139623264462656,MainProcess):2023-03-21-09:06:11.837.801 [mindspore/dataset/core/validator_helpers.py:804] 'Compose' from mindspore.dataset.transforms.py_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Compose' from mindspore.dataset.transforms instead.
[WARNING] ME(10084:139623264462656,MainProcess):2023-03-21-09:06:13.368.688 [mindspore/dataset/engine/datasets_user_defined.py:766] GeneratorDataset's num_parallel_workers: 8 is too large which may cause a lot of memory occupation (>85%) or out of memory(OOM) during multiprocessing. Therefore, it is recommended to reduce num_parallel_workers to 1 or smaller.
[WARNING] ME(10084:139623264462656,MainProcess):2023-03-21-09:06:15.598.997 [mindspore/dataset/core/validator_helpers.py:804] 'Compose' from mindspore.dataset.transforms.py_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'Compose' from mindspore.dataset.transforms instead.
[WARNING] ME(

eval: 1/832
eval: 2/832
eval: 3/832
eval: 4/832


KeyboardInterrupt: 

## Code

[Gitee](https://gitee.com/yanlq46462828/zjut_mindvideo)

[GitHub](https://github.com/STONE-SAMA/x3d_mindspore)
